In [1]:
import os

import dolfinx
import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pyvista
import pyvista as pv
import pyvistaqt as pvqt
import ufl
import warnings

from dolfinx import cpp, default_real_type, default_scalar_type, fem, io, la, mesh, nls, plot
from dolfinx.fem import petsc
from dolfinx.io import gmshio, VTXWriter
from dolfinx.nls import petsc as petsc_nls
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells
from IPython.display import Image

from mpi4py import MPI
from petsc4py import PETSc
from ufl import (Circumradius, FacetNormal, SpatialCoordinate, TrialFunction, TestFunction,
                 dot, div, dx, ds, dS, grad, inner, grad, avg, jump)

import utils

In [2]:
class Markers:
    def __init__(self):
        pass
    @property
    def left(self):
        return 1
    
    @property
    def right(self):
        return 2
    
    @property
    def domain(self):
        return 1

workdir = "output/poisson_nernst_planck"
utils.make_dir_if_missing(workdir)
output_meshfile = os.path.join(workdir, "mesh.msh")
point_0 = (0, 0, 0)
point_1 = (150e-6, 0, 0)
markers = Markers()
gmsh.initialize()
gmsh.model.add('interval')
p0 = gmsh.model.occ.addPoint(*point_0)
p1 = gmsh.model.occ.addPoint(*point_1)
interval = gmsh.model.occ.addLine(p0, p1)
gmsh.model.occ.synchronize()
gmsh.model.addPhysicalGroup(0, [p0], markers.left, "left")
gmsh.model.addPhysicalGroup(0, [p1], markers.right, "right")
gmsh.model.addPhysicalGroup(1, [interval], markers.domain, "interval")
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(1)
gmsh.write(output_meshfile)
gmsh.finalize()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 7.774e-05s, CPU 7.3e-05s)
Info    : 6 nodes 7 elements
Info    : Writing 'output/poisson_nernst_planck/mesh.msh'...
Info    : Done writing 'output/poisson_nernst_planck/mesh.msh'


In [3]:
D_n = 1e-10
D_p = 1e-10
faraday_const = 96485
R = 8.3145
T = 298

In [4]:
comm = MPI.COMM_WORLD
partitioner = mesh.create_cell_partitioner(mesh.GhostMode.shared_facet)
domain, ct, ft = gmshio.read_from_msh(output_meshfile, comm, partitioner=partitioner)
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(tdim, fdim)

Info    : Reading 'output/poisson_nernst_planck/mesh.msh'...
Info    : 3 entities
Info    : 6 nodes
Info    : 7 elements
Info    : Done reading 'output/poisson_nernst_planck/mesh.msh'


In [5]:
V = dolfinx.fem.functionspace(domain, ("CG", 1, (3,), ))
u_t = dolfinx.fem.Function(V) # previous step.
u = dolfinx.fem.Function(V) # current step.
u_t_n, u_t_p, phi_t = ufl.split(u_t)
u_n, u_p, phi = ufl.split(u)
v_n, v_p, v_phi = ufl.TestFunction(V)

In [6]:
# initial concentrations
u_t.sub(0).interpolate(lambda x: 1e04 + x[0] - x[0])
u_t.sub(1).interpolate(lambda x: 1e04 + x[0] - x[0])
# initial potential 
u_t.sub(2).interpolate(lambda x: x[0] / 150e-6)
u_t.x.scatter_forward()

## Poisson-Nernst-Planck Equation
The flux of species $i$ is given by the Nernst-Planck equation
$$\pmb{N}_i = -z_i u_i F \nabla \phi - D_i \nabla c_i + c_i \pmb{v}$$
Material balance is given by
$$\dfrac{\partial c_i}{\partial t} = -\nabla \cdot \pmb{N}_i + \mathcal{R}_i$$
where $\mathcal{R}_i$ is the homogeneous reaction.

The relationship between diffusivity $D_i$ and mobility $u_i$ is given by the Nernst-Einstein relationship
$$D_i = R T u_i$$

To solve the Nernst-Planck equation for two charged species, we need three test functions and three trial functions---one for +ve species, one for -ve species and one for potential.

An additional equation that has to be included is the charge conservation equation in the form of Poisson equation, to give the pair of equations called the Poisson-Nernst-Planck equation.
$$\nabla \cdot (-\kappa\nabla\phi)=0$$

In battery systems, one can ignore the advection term $c_i\pmb{v}$ if the system is not driven by bulk flow. Such battery systems include Li-ion batteries.
### Boundary and Initial Conditions
- initial concentration of both +ve and -ve species must be given
- initial potential gradient must be provided
- normal flux of -ve species is zero on the external boundary $\partial \Omega$
- normal flux of +ve species, assumed to be Li$^+$, is zero on insulated boundary and non-zero on the boundaries where charge transfer occurs at
- normal gradient of potential is zero on the insulated boundaries
- normal gradient of potential and normal flux of Li$^+$ are coupled at the charge transfer boundaries